# Exercise Instructions: Panel Data Modeling with Machine Learning Models

**Objective:**
The goal of this exercise is to practice panel data modeling skills using three machine learning models (Random Forest, Single Decision Tree, and Linear Regression with Elastic Net) that have not been utilized in the project so far. Completing the entire task or a significant portion during the class will earn you an additional 7 points (above what is outlined in the syllabus) towards your final grade.

**Tasks:**

1. **GitHub Setup:**
   - If you haven't done so already, [create](https://github.com/join) a GitHub account.
   - [Download](https://desktop.github.com) and [configure](https://docs.github.com/en/desktop/configuring-and-customizing-github-desktop/configuring-basic-settings-in-github-desktop) GitHub Desktop on your laptop. (Here you can find nice intro to the GitHub Dekstop app: [link](https://joshuadull.github.io/GitHub-Desktop/02-getting-started/index.html)). If you prefare git command line usage you can go with this [instruction](https://github.com/michaelwozniak/ml2_tools?tab=readme-ov-file#git).
2. **Repository Forking:**
   - [Fork](https://docs.github.com/en/pull-requests/collaborating-with-pull-requests/working-with-forks/fork-a-repo) the following repository to your projects: [https://github.com/michaelwozniak/ML-in-Finance-I-case-study-forecasting-tax-avoidance-rates](https://github.com/michaelwozniak/ML-in-Finance-I-case-study-forecasting-tax-avoidance-rates)

3. **Repository Cloning:**
   - [Clone](https://docs.github.com/en/desktop/adding-and-cloning-repositories/cloning-a-repository-from-github-to-github-desktop) the forked repository to your local computer using GitHub Desktop.

4. **Notebook Exploration:**
   - Open the file `notebooks/10.exercise.ipynb` to begin the ML tasks.

5. **Model Creation:**

   In the file `notebooks/10.exercise.ipynb`:
   - Create the following models:
      1. Random Forest ([RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html))
      2. Decision Tree ([DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html))
      3. Linear Regression with Elastic Net ([ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html))
   
   Follow a similar process to the models presented in class (e.g., KNN - `notebooks/07.knn-model.ipynb`):
      - Load the prepared training data.
      - Perform feature engineering if deemed necessary (note: these three models do not require data standardization, unlike SVM and KNN).
      - Conduct feature selection.
      - Perform hyperparameter tuning.
      - Identify a local champion for each model class (the best model for RF, DT, Elastic Net).
      - Save local champions to a pickle file.

6. **Model Evaluation:**
   - In the notebook `notebooks/09.final-comparison-and-summary.ipynb`, load the models you created and check if they outperform the previously used models.

7. **Version Control:**
   - At the end of the class, even if the tasks are incomplete, [commit](https://docs.github.com/en/desktop/making-changes-in-a-branch/committing-and-reviewing-changes-to-your-project-in-github-desktop) your changes using GitHub Desktop.
   - [Push](https://docs.github.com/en/desktop/making-changes-in-a-branch/pushing-changes-to-github-from-github-desktop) your changes to your remote GitHub repository.

8. **Submission:**
   - Send me the link to your GitHub project (my email: *mj.wozniak9@uw.edu.pl*).

Good luck with the exercise! If you have any questions, feel free to ask.

In [1]:
# Load Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_validate
from sklearn.metrics import mean_squared_error, accuracy_score, make_scorer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.inspection import permutation_importance
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from ReliefF import ReliefF
import pickle

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 150)


In [3]:
preprocessed_output_data_path = "../data/output"

df = pd.read_csv(f"{preprocessed_output_data_path}/train_fe.csv", index_col=0)

fr = pd.read_excel(f"{preprocessed_output_data_path}/feature_ranking.xlsx", index_col=0)

# Feature Engineering 

In [7]:
columns = df.columns.tolist()
# Because features with more than 2 unique values behave like continuous variables, so they benefit from standardization.
standardization = list()
not_standardization = list()
for i in columns:
    if df[i].nunique() > 2:
        standardization.append(i)
    else:
        not_standardization.append(i)
print(standardization)

['Ticker', 'Nazwa2', 'rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'etr', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'capex2_scaled', 'y_v2x_polyarchy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'ta_log', 'ppent_sqrt', 'intant_sqrt', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'rd_sqrt', 'ppe_clip', 'cash_holdings_sqrt', 'diff_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']


In [ ]:
standardization.remove("etr")
standardization.remove("Ticker")
standardization.remove("Nazwa2")
standardization.append("y_e_p_polity")
print(standardization)

['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'capex2_scaled', 'y_v2x_polyarchy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'ta_log', 'ppent_sqrt', 'intant_sqrt', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'rd_sqrt', 'ppe_clip', 'cash_holdings_sqrt', 'diff_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past', 'y_e_p_polity']


In [14]:
scaler = MinMaxScaler()
scaler.fit(df[standardization])
df[standardization] = scaler.transform(df[standardization])

In [15]:
pickle.dump(scaler, open("../models_shah/minmaxscaler.sav", "wb"))

In [16]:
var = fr.mi_score.sort_values(ascending=False).index.tolist()[0:10]

In [17]:
print(var)

['etr_y_past', 'etr_y_ma', 'txt', 'diff', 'ni', 'pi', 'intant', 'intant_sqrt', 'ta', 'revenue']


In [18]:
# sq root of number of rows | Rule-of-thumb sample size for choosing k nearest neighbors in feature selection / model complexity
df.shape[0] ** (0.5)

63.190189111918315

In [ ]:
# Intital Grid search for hyper parameters
param = {
    "n_estimators": [100, 200, 300], # 200 should be enough trees for stability; still fast with 63 features.
    "max_depth": [None, 5, 10, 20],  # 10 prevents overfitting; 63 features don’t need very deep trees.
    "min_samples_split": [2, 5, 10], # 5  Adds regularization; to avoid noisy tiny splits with many features
    "min_samples_leaf": [1, 2, 4], # 2  Smooths predictions; reduce variance from too-small leaves.
    "max_features": ["sqrt", "log2"], # Standard best default; √63 ≈ 7.9 features per split = good decorrelation.
}


In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np
import pandas as pd


In [21]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

model = RandomForestRegressor()
grid_CV = GridSearchCV(
    model, param, cv=5, scoring=mse, return_train_score=True, n_jobs=-1
)
grid_CV.fit(df.loc[:, var].values, df.loc[:, "etr"].values.ravel())

grid_CV.best_params_



{'max_depth': 5,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 300}

# Feature selection
We base our feature selection on the comparison of the following methods: 
- Benchmarks : Directly from 07 knn-model.ipynb.
- Feature ranking : We use Boruta and sorted mi .
- Highest correlation (top 20) of the features to target.
- Forward elimination : add features step-by-step using RF as the evaluator.
- Relief-style idea : same logic but use RF distances/splits: keep features that improve split quality the most.

In [38]:
benchmark = [
    "rok",
    "ta",
    "txt",
    "pi",
    "str",
    "xrd",
    "ni",
    "ppent",
    "intant",
    "dlc",
    "dltt",
    "capex",
    "revenue",
    "cce",
    "adv",
    "diff",
    "roa",
    "lev",
    "intan",
    "rd",
    "ppe",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
    "capex2_scaled",
    "y_v2x_polyarchy",
    "y_e_p_polity",
    "y_BR_Democracy",
    "WB_GDPgrowth",
    "WB_GDPpc",
    "WB_Inflation",
    "rr_per_country",
    "rr_per_sector",
    "sektor_consumer discretionary",
    "sektor_consumer staples",
    "sektor_energy",
    "sektor_health care",
    "sektor_industrials",
    "sektor_materials",
    "sektor_real estate",
    "sektor_technology",
    "sektor_utilities",
    "gielda_2",
    "gielda_3",
    "gielda_4",
    "gielda_5",
    "etr_y_past",
    "etr_y_ma",
    "diff_ma",
    "roa_ma",
    "lev_ma",
    "intan_ma",
    "ppe_ma",
    "sale_ma",
    "cash_holdings_ma",
    "roa_past",
    "lev_past",
    "intan_past",
    "ppe_past",
    "sale_past",
    "cash_holdings_past",
]

In [27]:
benchmark2 = [
    "ta",
    "txt",
    "pi",
    "str",
    "xrd",
    "ni",
    "ppent",
    "intant",
    "dlc",
    "dltt",
    "capex",
    "revenue",
    "cce",
    "adv",
    "diff",
    "roa",
    "lev",
    "intan",
    "rd",
    "ppe",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
    "y_v2x_polyarchy",
    "WB_GDPgrowth",
    "WB_GDPpc",
    "WB_Inflation",
    "rr_per_country",
    "rr_per_sector",
    "etr_y_past",
    "etr_y_ma",
    "diff_ma",
    "roa_ma",
    "lev_ma",
    "intan_ma",
    "ppe_ma",
    "sale_ma",
    "cash_holdings_ma",
    "roa_past",
    "lev_past",
    "intan_past",
    "ppe_past",
    "sale_past",
    "cash_holdings_past",
]

In [23]:
forward_elimination = [
    "rok",
    "ta",
    "txt",
    "pi",
    "str",
    "xrd",
    "ni",
    "ppent",
    "intant",
    "dlc",
    "dltt",
    "capex",
    "revenue",
    "cce",
    "adv",
    "diff",
    "roa",
    "lev",
    "intan",
    "rd",
    "ppe",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
    "capex2_scaled",
    "y_v2x_polyarchy",
    "y_e_p_polity",
    "y_BR_Democracy",
    "WB_GDPgrowth",
    "WB_GDPpc",
    "WB_Inflation",
    "rr_per_country",
    "rr_per_sector",
    "sektor_consumer discretionary",
    "sektor_consumer staples",
    "sektor_energy",
    "sektor_health care",
    "sektor_industrials",
    "sektor_materials",
    "sektor_real estate",
    "sektor_technology",
    "sektor_utilities",
    "gielda_2",
    "gielda_3",
    "gielda_4",
    "gielda_5",
    "ta_log",
    "txt_cat_(-63.011, -34.811]",
    "txt_cat_(-34.811, 0.488]",
    "txt_cat_(0.488, 24.415]",
    "txt_cat_(24.415, 25.05]",
    "txt_cat_(25.05, 308.55]",
    "txt_cat_(308.55, 327.531]",
    "txt_cat_(327.531, inf]",
    "pi_cat_(-8975.0, -1.523]",
    "pi_cat_(-1.523, 157.119]",
    "pi_cat_(157.119, 465.9]",
    "pi_cat_(465.9, 7875.5]",
    "pi_cat_(7875.5, 8108.5]",
    "pi_cat_(8108.5, inf]",
    "str_cat_(0.0875, 0.192]",
    "str_cat_(0.192, 0.28]",
    "str_cat_(0.28, inf]",
    "xrd_exists",
    "ni_profit",
    "ni_profit_20000",
    "ppent_sqrt",
    "intant_sqrt",
    "dlc_cat_(42.262, 176.129]",
    "dlc_cat_(176.129, 200.9]",
    "dlc_cat_(200.9, inf]",
    "dltt_cat_(39.38, 327.85]",
    "dltt_cat_(327.85, 876.617]",
    "dltt_cat_(876.617, inf]",
    "capex_cat_(7.447, 79.55]",
    "capex_cat_(79.55, 5451.0]",
    "capex_cat_(5451.0, inf]",
    "revenue_cat_(0.174, 1248.817]",
    "revenue_cat_(1248.817, 4233.587]",
    "revenue_cat_(4233.587, inf]",
    "cce_cat_(5.619, 63.321]",
    "cce_cat_(63.321, inf]",
    "adv_cat_(0.3, 874.5]",
    "adv_cat_(874.5, inf]",
    "diff_positive",
    "roa_clip",
    "lev_sqrt",
    "intan_pow2",
    "rd_sqrt",
    "ppe_clip",
    "cash_holdings_sqrt",
    "adv_expenditure_positive",
    "diff_dta",
    "cfc_dta",
    "etr_y_past",
    "etr_y_ma",
    "diff_ma",
    "roa_ma",
    "lev_ma",
    "intan_ma",
    "ppe_ma",
    "sale_ma",
    "cash_holdings_ma",
    "roa_past",
    "lev_past",
    "intan_past",
    "ppe_past",
    "sale_past",
    "cash_holdings_past",
]
forward_elimination.remove("ta_log")
forward_elimination.remove("ppent_sqrt")
forward_elimination.remove("intant_sqrt")
forward_elimination.remove("roa")
forward_elimination.remove("lev")
forward_elimination.remove("intan")
forward_elimination.remove("rd_sqrt")
forward_elimination.remove("ppe")
forward_elimination.remove("cash_holdings_sqrt")

In [24]:
# Boruta 
br_features = fr[fr.boruta_rank.isin([1, 2, 3])].index.tolist()

In [ ]:
# mi features 
fr.sort_values("mi_score", ascending=False, inplace=True)
# top 20 features
mi_features = fr.iloc[0:20].index.tolist()
# top 25 features
mi_features_25 = fr.iloc[0:25].index.tolist()
# top 35 features
mi_features_35 = fr.iloc[0:35].index.tolist()

# top 50 features
mi_features_50 = fr.iloc[0:50].index.tolist()


In [26]:
# based on strongest correlations . Top 20 
fr["corr_abs"] = np.abs(fr["corr"])
fr.sort_values("corr_abs", ascending=False, inplace=True)
corr_features = fr.iloc[0:20].index.tolist()

In [31]:
# So we remove all binned features for this feature selection method.
candidates_without_discr = [i for i in forward_elimination if "]" not in i]

In [34]:
# Forward elimination feature set sf_features using only continuous features and sf_features2 including discritized features.

model = RandomForestRegressor(**grid_CV.best_params_, n_jobs=-1)

sf = SFS(
    model,
    k_features=(5, 15),
    forward=True,
    floating=False,
    verbose=0,
    scoring=mse,
    cv=5,
    n_jobs=-1,
)

# without binned/ discretized features
sffit1 = sf.fit(
    df.loc[:, candidates_without_discr].values, df.loc[:, "etr"].values.ravel()
)

sf_features = df.loc[:, candidates_without_discr].columns[list(sffit1.k_feature_idx_)]


# with binned/ discretized features
sffit2 = sf.fit(df.loc[:, forward_elimination].values, df.loc[:, "etr"].values.ravel())

sf_features2 = df.loc[:, forward_elimination].columns[list(sffit2.k_feature_idx_)]

print(sf_features)
print(sf_features2)

Index(['rok', 'cfc', 'dta', 'y_BR_Democracy', 'WB_GDPpc', 'WB_Inflation',
       'etr_y_past', 'etr_y_ma', 'diff_ma'],
      dtype='object')
Index(['rok', 'WB_GDPpc', 'txt_cat_(308.55, 327.531]',
       'str_cat_(0.0875, 0.192]', 'str_cat_(0.28, inf]',
       'capex_cat_(5451.0, inf]', 'adv_cat_(874.5, inf]', 'etr_y_past',
       'etr_y_ma', 'diff_ma'],
      dtype='object')


# HyperParameter Tuning for each group of features

In [ ]:
rf_param = {
    "n_estimators": [100, 200, 300], # 200 should be enough trees for stability; still fast with 63 features.
    "max_depth": [None, 5, 10, 20],  # 10 prevents overfitting; 63 features don’t need very deep trees.
    "min_samples_split": [2, 5, 10], # 5  Adds regularization; to avoid noisy tiny splits with many features
    "min_samples_leaf": [1, 2, 4], # 2  Smooths predictions; reduce variance from too-small leaves.
    "max_features": ["sqrt", "log2"], # Standard best default; √63 ≈ 7.9 features per split = good decorrelation.
}


In [36]:

def cv_proc(var):
    model = RandomForestRegressor(random_state=11, n_jobs=-1)
    grid_CV = GridSearchCV( model, rf_param, cv=5, scoring=mse, return_train_score=True, n_jobs=-1)
    grid_CV.fit(df.loc[:, var].values, df.loc[:, "etr"].values.ravel())
    print(grid_CV.best_params_)
    print(grid_CV.best_score_)


In [ ]:
cv_proc(benchmark)

{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
-0.019768508497483732


In [40]:
cv_proc(mi_features)


{'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
-0.02002631534740848


In [41]:
cv_proc(mi_features_25)

{'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
-0.02007478974738903


In [42]:
cv_proc(mi_features_35)

{'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
-0.020229398635747447


In [43]:
cv_proc(mi_features_50)

{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 300}
-0.019886968022118177


In [44]:
cv_proc(br_features)

{'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
-0.01944825835771664


In [45]:
cv_proc(corr_features)

{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
-0.019897921565460122


In [ ]:
cv_proc(sf_features)



{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
-0.018731234351406383


In [58]:
cv_proc(sf_features2)

{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
-0.018618772143423334


## The 7 feature sets we will be testing for all three models 
- benchmark
- mi_features
- mi_features_25
- mi_features_35
- mi_features_50
- br_features
- corr_features
- sf_features
- sf_features2

In [56]:
print(f"Now we have shape {df.shape}")
print(f"Out of the 3993 rows we take the first train/validation split 1452 + 363")
print(f"That is 1815 observations")
print(f"Now we can do this twice, then we have exactly 363 left for final test")
print(f"So we need for i in range 6, because 3993/6 = 363")


Now we have shape (3993, 115)
Out of the 3993 rows we take the first train/validation split 1452 + 363
That is 1815 observations
Now we can do this twice, then we have exactly 363 left for final test
So we need for i in range 6, because 3993/6 = 363


In [69]:
def proper_CV(x,y, model, display_res=False):
    train_score = list()
    valid_score = list()
    train_indexes = [0, 1452]
    valid_indexes = [1452, 1815]
    for i in range(0,6):
        train_x = x[x.index.isin(range(train_indexes[0], train_indexes[1]))]
        train_y = y[y.index.isin(range(train_indexes[0], train_indexes[1]))]
        valid_x = x[x.index.isin(range(valid_indexes[0], valid_indexes[1]))]
        valid_y = y[y.index.isin(range(valid_indexes[0], valid_indexes[1]))]
        
        model.fit(train_x.values, train_y.values.ravel())
        
        pred_y_train = model.predict(train_x.values)
        rmse = np.sqrt(mean_squared_error(train_y, pred_y_train))
        train_score.append(rmse)
        
        pred_y_val = model.predict(valid_x.values)
        rmse = np.sqrt(mean_squared_error(valid_y, pred_y_val))
        valid_score.append(rmse)
        
        train_indexes = [0, valid_indexes[1]]
        valid_indexes = [train_indexes[1], valid_indexes[1] + 363]
        
    if display_res == True:
        view = pd.DataFrame([train_score, valid_score]).T.rename(
            columns= {0 : "cv_train", 1 : "cv_val"}
        )
        display(view)
        return train_score, valid_score, view
    else:
        return train_score, valid_score

In [70]:
hp_rf = [
    {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300},
    {'max_depth': 5,  'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100},
    {'max_depth': 5,  'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300},
    {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300},
    {'max_depth': None,'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 5,  'n_estimators': 300},
    {'max_depth': 5,  'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 2,  'n_estimators': 200},
    {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200},
    {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2,  'n_estimators': 300},
    {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2,  'n_estimators': 300},
]


In [ ]:
model = RandomForestRegressor(**hp_rf[0], n_jobs=-1, random_state=11)
var = benchmark
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.091265,0.154857
1,0.094208,0.121510
2,0.093138,0.141350
3,0.094701,0.151206
4,0.096161,0.143424
5,0.096512,0.156903


In [75]:
model = RandomForestRegressor(**hp_rf[1], n_jobs=-1, random_state=11)
var = mi_features
cv_output1 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.112158,0.154515
1,0.118160,0.121557
2,0.118387,0.142462
3,0.120166,0.150319
4,0.123174,0.145488
5,0.125394,0.157527


In [76]:
model = RandomForestRegressor(**hp_rf[2], n_jobs=-1, random_state=11)
var = mi_features_25
cv_output2 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.111444,0.155612
1,0.118248,0.122082
2,0.118392,0.142977
3,0.120612,0.151773
4,0.123597,0.145695
5,0.125483,0.157518


In [ ]:
model = RandomForestRegressor(**hp_rf[3], n_jobs=-1, random_state=11)
var = mi_features_35
cv_output3 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.092427,0.155103
1,0.095754,0.121784
2,0.094312,0.143761
3,0.096145,0.153250
4,0.098045,0.145674
5,0.097978,0.157716


In [78]:
model = RandomForestRegressor(**hp_rf[4], n_jobs=-1, random_state=11)
var = mi_features_50
cv_output4 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.088563,0.153634
1,0.091632,0.121804
2,0.090458,0.142179
3,0.091702,0.151995
4,0.093034,0.143695
5,0.093221,0.156489


In [ ]:
model = RandomForestRegressor(**hp_rf[5], n_jobs=-1, random_state=11)
var = br_features
cv_output5 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.111891,0.153038
1,0.118263,0.120002
2,0.117696,0.140438
3,0.119459,0.149293
4,0.122459,0.143143
5,0.124006,0.153161


In [80]:
model = RandomForestRegressor(**hp_rf[6], n_jobs=-1, random_state=11)
var = corr_features
cv_output6 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.098978,0.157742
1,0.103144,0.121161
2,0.102941,0.139570
3,0.104449,0.150442
4,0.107729,0.144278
5,0.108974,0.155627


In [81]:
model = RandomForestRegressor(**hp_rf[7], n_jobs=-1, random_state=11)
var = sf_features
cv_output7 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.086539,0.153824
1,0.090203,0.120188
2,0.090617,0.140585
3,0.092714,0.149203
4,0.095919,0.137684
5,0.097460,0.151078


In [82]:
model = RandomForestRegressor(**hp_rf[8], n_jobs=-1, random_state=11)
var = sf_features2
cv_output8 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.070096,0.156546
1,0.075943,0.119951
2,0.077872,0.137611
3,0.081239,0.145954
4,0.086050,0.136821
5,0.089181,0.152711


In [87]:
dfs = [cv_output0, cv_output1, cv_output2, cv_output3,
       cv_output4, cv_output5, cv_output6, cv_output7, cv_output8]

val_means = [np.mean(o[1]) for o in dfs]          # average cv_val for each model
best_idx = int(np.argmin(val_means))             # helper: index of best model

best_model = dfs[best_idx]                       # whole tuple for best model
best_view = best_model[2]                        # the table (cv_train, cv_val)

print("Best model index:", best_idx)
best_view


Best model index: 8


,cv_train,cv_val
0,0.070096,0.156546
1,0.075943,0.119951
2,0.077872,0.137611
3,0.081239,0.145954
4,0.086050,0.136821
5,0.089181,0.152711


Model 8 seems to be the best one 
It uses the features from forward elimination with continuous features included.

In [88]:
print(sf_features2)

Index(['rok', 'WB_GDPpc', 'txt_cat_(308.55, 327.531]',
       'str_cat_(0.0875, 0.192]', 'str_cat_(0.28, inf]',
       'capex_cat_(5451.0, inf]', 'adv_cat_(874.5, inf]', 'etr_y_past',
       'etr_y_ma', 'diff_ma'],
      dtype='object')


# Final Model : Random Forest

In [100]:
model_rf = RandomForestRegressor(**hp_rf[8], n_jobs=-1, random_state=11)
model_rf.fit(df.loc[:, sf_features2].values, df.loc[:, "etr"].values.ravel())

,n_estimators,300
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [103]:
filename = "../models_shah/rf.sav"

In [104]:
# write binary , to load use rb "read binary"
pickle.dump(model_rf, open(filename, "wb"))

# Decision trees

In [161]:

tree_param = {
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"],
    "criterion": ["squared_error", "friedman_mse"],
    "max_leaf_nodes": [None, 50, 100]
}


In [159]:
from sklearn.tree import DecisionTreeRegressor

def cv_proc(var):
    model = DecisionTreeRegressor(random_state=11)
    grid_CV = GridSearchCV(
        model, tree_param, cv=5, scoring=mse, return_train_score=True, n_jobs=-1)
    grid_CV.fit(df.loc[:, var].values, df.loc[:, "etr"].values.ravel())
    print(grid_CV.best_params_)
    print(grid_CV.best_score_)


In [160]:
cv_proc(benchmark)

{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
-0.021231992596870437


In [162]:
cv_proc(mi_features)

{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 10}
-0.022354691219181023


In [163]:
cv_proc(mi_features_25)

{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 50, 'min_samples_leaf': 4, 'min_samples_split': 2}
-0.02157270452821462


In [164]:
cv_proc(mi_features_35)

{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 5}
-0.022309246117300098


In [165]:
cv_proc(mi_features_50)

{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 10}
-0.02174688966240417


In [166]:
cv_proc(br_features)


{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
-0.021180043511888354


In [167]:
cv_proc(corr_features)

{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
-0.0208451825464382


In [168]:
cv_proc(sf_features)

{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
-0.020000892641804883


In [169]:
cv_proc(sf_features2)

{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': 50, 'min_samples_leaf': 4, 'min_samples_split': 10}
-0.020421579029444692


In [171]:
def proper_CV(x,y, model, display_res=False):
    train_score = list()
    valid_score = list()
    train_indexes = [0, 1452]
    valid_indexes = [1452, 1815]
    for i in range(0,6):
        train_x = x[x.index.isin(range(train_indexes[0], train_indexes[1]))]
        train_y = y[y.index.isin(range(train_indexes[0], train_indexes[1]))]
        valid_x = x[x.index.isin(range(valid_indexes[0], valid_indexes[1]))]
        valid_y = y[y.index.isin(range(valid_indexes[0], valid_indexes[1]))]
        
        model.fit(train_x.values, train_y.values.ravel())
        
        pred_y_train = model.predict(train_x.values)
        rmse = np.sqrt(mean_squared_error(train_y, pred_y_train))
        train_score.append(rmse)
        
        pred_y_val = model.predict(valid_x.values)
        rmse = np.sqrt(mean_squared_error(valid_y, pred_y_val))
        valid_score.append(rmse)
        
        train_indexes = [0, valid_indexes[1]]
        valid_indexes = [train_indexes[1], valid_indexes[1] + 363]
        
    if display_res == True:
        view = pd.DataFrame([train_score, valid_score]).T.rename(
            columns= {0 : "cv_train", 1 : "cv_val"}
        )
        display(view)
        return train_score, valid_score, view
    else:
        return train_score, valid_score

In [172]:
hp_tree = [
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 2},
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 10},
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 50, 'min_samples_leaf': 4, 'min_samples_split': 2},
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 5},
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_samples_leaf': 2, 'min_samples_split': 10},
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 2},
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 2},
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 2},
{'criterion': 'squared_error', 'max_depth': 5, 'max_features': 'sqrt', 'max_leaf_nodes': 50, 'min_samples_leaf': 4, 'min_samples_split': 10}
]

In [173]:
model = DecisionTreeRegressor(**hp_tree[0], random_state=11)
var = benchmark
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.115316,0.161087
1,0.119839,0.142618
2,0.122295,0.160257
3,0.120404,0.157119
4,0.129390,0.150675
5,0.124719,0.164442


In [174]:
model = DecisionTreeRegressor(**hp_tree[1], random_state=11)
var = mi_features
cv_output1 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.118364,0.170776
1,0.128460,0.135619
2,0.123704,0.157535
3,0.129510,0.169037
4,0.131502,0.142603
5,0.131756,0.169185


In [175]:
model = DecisionTreeRegressor(**hp_tree[2], random_state=11)
var = mi_features_25
cv_output2 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.117547,0.169144
1,0.126461,0.133744
2,0.123526,0.154578
3,0.126680,0.154800
4,0.130046,0.150270
5,0.131615,0.169415


In [176]:
model = DecisionTreeRegressor(**hp_tree[3], random_state=11)
var = mi_features_35
cv_output3 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.113029,0.172512
1,0.124391,0.137176
2,0.120162,0.151065
3,0.126541,0.172495
4,0.127595,0.152430
5,0.128145,0.164918


In [177]:
model = DecisionTreeRegressor(**hp_tree[4], random_state=11)
var = mi_features_50
cv_output4 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.120225,0.164729
1,0.125040,0.159638
2,0.126663,0.154855
3,0.128936,0.163037
4,0.131534,0.147822
5,0.133351,0.162873


In [178]:
model = DecisionTreeRegressor(**hp_tree[5], random_state=11)
var = br_features
cv_output5 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.115354,0.158497
1,0.122493,0.125270
2,0.121699,0.144306
3,0.124649,0.159406
4,0.126921,0.143071
5,0.129436,0.156323


In [179]:
model = DecisionTreeRegressor(**hp_tree[6], random_state=11)
var = corr_features
cv_output6 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.114926,0.170542
1,0.126047,0.123879
2,0.122677,0.151623
3,0.125976,0.152297
4,0.128697,0.151908
5,0.129692,0.157531


In [180]:
model = DecisionTreeRegressor(**hp_tree[7], random_state=11)
var = sf_features
cv_output7 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.116019,0.162936
1,0.123184,0.131914
2,0.122963,0.142941
3,0.125721,0.152510
4,0.129453,0.145779
5,0.129451,0.159571


In [181]:
model = DecisionTreeRegressor(**hp_tree[8], random_state=11)
var = sf_features2
cv_output8 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.118407,0.157001
1,0.125732,0.126831
2,0.123232,0.148373
3,0.126960,0.152191
4,0.130600,0.144860
5,0.130288,0.157713


In [182]:
# Best Model based on Average of cv_val results

dfs = [cv_output0, cv_output1, cv_output2, cv_output3,
       cv_output4, cv_output5, cv_output6, cv_output7, cv_output8]

val_means = [np.mean(o[1]) for o in dfs]          # average cv_val for each model
best_idx = int(np.argmin(val_means))             # helper: index of best model

best_model = dfs[best_idx]                       # whole tuple for best model
best_view = best_model[2]                        # the table (cv_train, cv_val)

print("Best model index:", best_idx)
best_view


Best model index: 5


,cv_train,cv_val
0,0.115354,0.158497
1,0.122493,0.125270
2,0.121699,0.144306
3,0.124649,0.159406
4,0.126921,0.143071
5,0.129436,0.156323


# Final Model : Decision Trees

In [183]:
model_tree = RandomForestRegressor(**hp_rf[5], n_jobs=-1, random_state=11)
model_tree.fit(df.loc[:, br_features].values, df.loc[:, "etr"].values.ravel())

,n_estimators,200
,criterion,'squared_error'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,4
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [184]:
filename = "../models_shah/tree.sav"
# write binary , to load use rb "read binary"
pickle.dump(model_rf, open(filename, "wb"))


# Linear Regression Elastic Net

In [188]:
en_param = {
    "alpha": [0.001, 0.01, 0.1, 1],
    "l1_ratio": [0.1, 0.5, 0.9]
}


In [187]:
from sklearn.linear_model import ElasticNet

def cv_proc(var):
    model = ElasticNet(random_state=11)
    grid_CV = GridSearchCV(model, en_param, cv=5, scoring=mse, return_train_score=True, n_jobs=-1)

    grid_CV.fit(df.loc[:, var].values, df.loc[:, "etr"].values.ravel())
    print(grid_CV.best_params_)
    print(grid_CV.best_score_)


In [190]:
cv_proc(benchmark)

{'alpha': 0.001, 'l1_ratio': 0.5}
-0.020184158729257447


In [191]:
cv_proc(mi_features)

{'alpha': 0.001, 'l1_ratio': 0.1}
-0.02037727414053621


In [192]:
cv_proc(mi_features_25)

{'alpha': 0.001, 'l1_ratio': 0.1}
-0.020378247678047433


In [193]:
cv_proc(mi_features_35)

{'alpha': 0.001, 'l1_ratio': 0.1}
-0.020364199531972094


In [194]:
cv_proc(mi_features_50)

{'alpha': 0.001, 'l1_ratio': 0.5}
-0.020227521148681926


In [195]:
cv_proc(br_features)

{'alpha': 0.001, 'l1_ratio': 0.1}
-0.020169872471706568


In [196]:
cv_proc(corr_features)

{'alpha': 0.001, 'l1_ratio': 0.1}
-0.020115813155580996


In [197]:
cv_proc(sf_features)

{'alpha': 0.001, 'l1_ratio': 0.1}
-0.02019083300144895


In [198]:
cv_proc(sf_features2)

{'alpha': 0.001, 'l1_ratio': 0.1}
-0.019990971284128956


In [200]:
hp_en = [
{'alpha': 0.001, 'l1_ratio': 0.5},
{'alpha': 0.001, 'l1_ratio': 0.1},
{'alpha': 0.001, 'l1_ratio': 0.1},
{'alpha': 0.001, 'l1_ratio': 0.1},
{'alpha': 0.001, 'l1_ratio': 0.5},
{'alpha': 0.001, 'l1_ratio': 0.1},
{'alpha': 0.001, 'l1_ratio': 0.1},
{'alpha': 0.001, 'l1_ratio': 0.1},
{'alpha': 0.001, 'l1_ratio': 0.1}
]

In [201]:
from sklearn.linear_model import ElasticNet

model = ElasticNet(**hp_en[0], random_state=11)
var = benchmark
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.127042,0.147377
1,0.131250,0.122593
2,0.129960,0.146735
3,0.132516,0.152203
4,0.134950,0.144726
5,0.136029,0.159134


In [202]:
model = ElasticNet(**hp_en[1], random_state=11)
var = mi_features
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.128139,0.149026
1,0.132399,0.122579
2,0.130900,0.148076
3,0.133450,0.151580
4,0.135747,0.144852
5,0.136706,0.161069


In [203]:

model = ElasticNet(**hp_en[2], random_state=11)
var = mi_features_25
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.128071,0.148879
1,0.132279,0.122741
2,0.130842,0.148112
3,0.133419,0.151644
4,0.135722,0.144786
5,0.136659,0.161043


In [204]:

model = ElasticNet(**hp_en[3], random_state=11)
var = mi_features_35
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.127816,0.149280
1,0.132091,0.122523
2,0.130654,0.148514
3,0.133245,0.151671
4,0.135574,0.144326
5,0.136449,0.160873


In [205]:

model = ElasticNet(**hp_en[4], random_state=11)
var = mi_features_50
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.127553,0.148143
1,0.131874,0.123015
2,0.130543,0.146609
3,0.133040,0.152161
4,0.135489,0.144187
5,0.136509,0.159469


In [206]:

model = ElasticNet(**hp_en[5], random_state=11)
var = br_features
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.128801,0.147094
1,0.132700,0.122143
2,0.130782,0.147727
3,0.133279,0.151981
4,0.135479,0.143857
5,0.136190,0.159368


In [207]:

model = ElasticNet(**hp_en[6], random_state=11)
var = corr_features
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.127065,0.149128
1,0.131525,0.121987
2,0.129959,0.146339
3,0.132378,0.151385
4,0.134804,0.144286
5,0.135814,0.159579


In [208]:

model = ElasticNet(**hp_en[7], random_state=11)
var = sf_features
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.128656,0.147505
1,0.132567,0.122376
2,0.130940,0.147198
3,0.133313,0.152037
4,0.135654,0.143974
5,0.136582,0.159163


In [209]:

model = ElasticNet(**hp_en[8], random_state=11)
var = sf_features2
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)


,cv_train,cv_val
0,0.127815,0.144873
1,0.131141,0.122420
2,0.129750,0.146110
3,0.132142,0.149150
4,0.134219,0.144542
5,0.135395,0.160900


In [210]:
# Best Model based on Average of cv_val results

dfs = [cv_output0, cv_output1, cv_output2, cv_output3,
       cv_output4, cv_output5, cv_output6, cv_output7, cv_output8]

val_means = [np.mean(o[1]) for o in dfs]          # average cv_val for each model
best_idx = int(np.argmin(val_means))             # helper: index of best model

best_model = dfs[best_idx]                       # whole tuple for best model
best_view = best_model[2]                        # the table (cv_train, cv_val)

print("Best model index:", best_idx)
best_view


Best model index: 0


,cv_train,cv_val
0,0.127815,0.144873
1,0.131141,0.122420
2,0.129750,0.146110
3,0.132142,0.149150
4,0.134219,0.144542
5,0.135395,0.160900


- benchmark
- mi_features
- mi_features_25
- mi_features_35
- mi_features_50
- br_features
- corr_features
- sf_features
- sf_features2

The benchmark features perform the best based on the average of the validation average.

In [211]:
from sklearn.linear_model import ElasticNet

model_en = ElasticNet(**hp_en[5], random_state=11)
model_en.fit(df.loc[:, br_features].values, df.loc[:, "etr"].values.ravel())


,alpha,0.001
,l1_ratio,0.1
,fit_intercept,True
,precompute,False
,max_iter,1000
,copy_X,True
,tol,0.0001
,warm_start,False
,positive,False
,random_state,11
,selection,'cyclic'


In [213]:
filename = "../models_shah/en.sav"
# write binary , to load use rb "read binary"
pickle.dump(model_en, open(filename, "wb"))
